In [ ]:
import os
import PIL 
import time 
import torch 
import torchvision 
import random

import numpy as np 
import torch.nn as nn 
import matplotlib.pyplot as plt 
import torch.nn.functional as F

#data transforms
from torch.autograd import Variable 
from torchvision import datasets, transforms 
from PIL import Image, ImageEnhance

#data aug
from augmentation.autoaugment import CIFAR10Policy 
from augmentation.cutout import Cutout 
from augmentation.AugMix.AugMix import AugMixDataset 
from augmentation.RandAugment import RandAugment

#optim and activation
from optim.deepmemory   import DeepMemory
from optim.lookahead    import Lookahead 
from optim.radam        import RAdam 
from adamod       import AdaMod
from activations  import Mish

from augmentation.ISDA import ISDALoss

from loss_func.cross_entropy import CrossEntropyLoss #https://github.com/eladhoffer/utils.pytorch

### Set Seed for Reproducibility

In [ ]:
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED']=str(seed)
    
set_seed(72)

### Preprocess and Load Data

In [ ]:
preprocess = transforms.Compose([
     transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (.2023, .1994, .2010)),  #CIFAR10
#      transforms.Normalize((0.507, 0.487, 0.441), (0.267, 0.256, 0.276))      #CIFAR100
#      Cutout(n_holes=1, length=16),               # CutOut
])

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4, fill=128),
#     CIFAR10Policy(),                    # AutoAugment
    preprocess

])

# train_transform.transforms.insert(0, RandAugment(1, 5))                    #RandAugment

test_transform = preprocess

# test_transform = transforms.Compose([
#     transforms.ToTensor(),
#      transforms.Normalize((0.4914, 0.4822, 0.4465), (.2023, .1994, .2010)),  #CIFAR10
#      transforms.Normalize((0.507, 0.487, 0.441), (0.267, 0.256, 0.276))      #CIFAR100
# ])

In [ ]:
batch_size = 4

train_data = datasets.CIFAR10(root="./data", train=True, download=True, transform=train_transform)

test_data= datasets.CIFAR10(root="./data", train=False, download=True, transform=test_transform)

# load training data in batches
train_loader = torch.utils.data.DataLoader(
#                       AugMixDataset(train_data, preprocess, no_jsd=True),      # Augmix
                      train_data,
                      batch_size=batch_size,
                      num_workers=8,
                      shuffle=True, 
                      pin_memory=True
                      )

# load test data in batches
test_loader = torch.utils.data.DataLoader(test_data,
                      batch_size=batch_size,
                      num_workers=8,
                      shuffle=False,
                      pin_memory=True
                      )

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
print(f'Length of train loader is {len(train_loader)}')
print(f'Length of test loader is {len(test_loader)}')

### Metrics

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

### Build Model

In [ ]:
class CustomModel(nn.Module):
    def __init__(self):        
        super(CustomModel, self).__init__()
        
        # Depthwise Convolutions
        self.layers = nn.Sequential(
                            nn.Conv2d(in_channels=1*3, out_channels=16*3, kernel_size=3, groups=1, stride=1, padding=1, bias=False),
                            Mish(),
                            nn.BatchNorm2d(num_features=16*3, eps=1e-3, momentum=0.99),
                                                      
                            nn.Conv2d(in_channels=16*3, out_channels=96, kernel_size=1, groups=8, stride=1, padding=0, bias=False),
                            Mish(),
                            nn.BatchNorm2d(num_features=96, eps=1e-3, momentum=0.99),
                                    
                            nn.Conv2d(in_channels=96, out_channels=128, kernel_size=3, groups=8, stride=1, padding=1, bias=False),
                            Mish(),
                            nn.BatchNorm2d(num_features=128, eps=1e-3, momentum=0.99),
                            nn.MaxPool2d(2, 2),
            
                            nn.Conv2d(in_channels=128, out_channels=192, kernel_size=1, groups=16, stride=1, padding=0, bias=False),
                            Mish(),
                            nn.BatchNorm2d(num_features=192, eps=1e-3, momentum=0.99),
                                                        
                            nn.Conv2d(in_channels=192, out_channels=256, kernel_size=3, groups=16, stride=1, padding=1, bias=False),
                            Mish(),
                            nn.BatchNorm2d(num_features=256, eps=1e-3, momentum=0.99),
                                                                                                                
                            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=1, groups=32, stride=1, padding=0, bias=False),
                            Mish(),
                            nn.BatchNorm2d(num_features=512, eps=1e-3, momentum=0.99),
                            nn.MaxPool2d(2, 2),
            
                            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, groups=64, stride=1, padding=1, bias=False),
                            Mish(),
                            nn.BatchNorm2d(num_features=512, eps=1e-3, momentum=0.99),
                                                        
                            nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1, groups=16, stride=1, padding=0, bias=False),
                            Mish(),
                            nn.BatchNorm2d(num_features=256, eps=1e-3, momentum=0.99),

                            nn.Conv2d(in_channels=256, out_channels=192, kernel_size=3, groups=16, stride=1, padding=1, bias=False),
                            Mish(),
                            nn.BatchNorm2d(num_features=192, eps=1e-3, momentum=0.99),
                            nn.MaxPool2d(2, 2),
                            )
        
        
        #squeeze and excitation
        self.se_reduce = nn.Conv2d(in_channels=192, out_channels=128, kernel_size=1)
        self.se_expand = nn.Conv2d(in_channels=128, out_channels=192, kernel_size=1)
        
        # fully connected layer
        # self.fc = nn.Linear(in_features=192*4*4, out_features=10)
        
        
    def forward(self, x):
        x = self.layers(x)
        x_squeezed = F.adaptive_avg_pool2d(x, x.size(2))
        x_squeezed = self.se_expand(Mish()(self.se_reduce(x_squeezed)))
        x = torch.sigmoid(x_squeezed) * x
        x = x.view(x.size(0), -1)
        # x = self.fc(x)

        return x
            
model = CustomModel()

In [ ]:
class Full_layer(torch.nn.Module):
    '''explicitly define the full connected layer'''

    def __init__(self, feature_num, class_num):
        super(Full_layer, self).__init__()
        self.class_num = class_num
        self.fc = nn.Linear(feature_num, class_num)

    def forward(self, x):
        x = self.fc(x)
        return x

In [ ]:
feature_num = 192*4*4
num_classes = len(classes)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model = torch.nn.DataParallel(model)
fc = Full_layer(feature_num, num_classes).to(device)
fc = torch.nn.DataParallel(fc)

In [ ]:
best_top1 = 0 # train from start
start_epoch = 0

In [ ]:
num_epochs = 50
combine_ratio = 0.5
isda_criterion = ISDALoss(feature_num, num_classes)
ce_criterion = CrossEntropyLoss(smooth_eps=0.1).to(device)
optimizer = Lookahead(DeepMemory([{'params': model.parameters()},
                             {'params': fc.parameters()}], len_memory=len(train_data.data)//batch_size))          # rectified adam wtih lookahead
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=len(train_data.data)//batch_size) 

In [ ]:
print('Number of model parameters: {}'.format(
        sum([p.data.nelement() for p in model.parameters()]) +
        sum([p.data.nelement() for p in fc.parameters()])
        ))

### Load Checkpoints to resume training

In [ ]:
# checkpoint = torch.load('./checkpoint/CustomModel_standard_dmla_isda_ckpt.pth')

# model.module.load_state_dict(checkpoint['model_state_dict'], strict=False)
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# fc.module.load_state_dict(checkpoint['fc'])
# start_epoch = checkpoint['epoch']
# best_loss = checkpoint['loss']
# best_top1 = checkpoint['top1']   # resume training
# best_top5 = checkpoint['top5']  

In [ ]:
# print(f'Top-1 Acc: {best_top1}\t Top-5 Acc: {best_top5}\t Best loss: {best_loss:.4f}\t Best epoch: {start_epoch}')
# print(f'Loaded checkpoint with \n {best_top1}% Top-1 Accuracy, {best_top5}% Top-5 Accuracy, after training for {start_epoch} epochs.')

### Train Model

In [ ]:
%%time
def train(train_loader, model, fc, criterion, optimizer, epoch):
    print('Training model...\n')
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    ratio = combine_ratio * (epoch / (num_epochs))
    
    # switch to train mode
    model.train()
    fc.train()

    end = time.time()
    
    for i, (input, target) in enumerate(train_loader):
                
        # measure data loading time
        data_time.update(time.time() - end)
        
        target = target.to(device)
        input_var = Variable(input)
        target_var = Variable(target)
        
        optimizer.zero_grad()
        
        # compute output
        loss, output = criterion(model, fc, input_var, target_var, ratio)
                  
        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target, topk=(1, 5))
        losses.update(loss.data.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))
        top5.update(prec5.item(), input.size(0))
       
        loss.backward()
        optimizer.step()
        
        lr_scheduler.step()
        
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 1500 == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Acc@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Acc@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                epoch, i, len(train_loader), batch_time=batch_time,
                data_time=data_time, loss=losses, top1=top1, top5=top5))
    
    print(' * Acc@1 {top1.avg:.3f} Acc@1 Error {top1_err:.3f}\n'
              ' * Acc@5 {top5.avg:.3f} Acc@5 Error {top5_err:.3f}'
              .format(top1=top1, top1_err=100-top1.avg, top5=top5, top5_err=100-top5.avg))

### Accuracy on test data

In [ ]:
%%time
def validate(test_loader, model, fc, criterion, epoch):
    print('Evaluating model on test data...\n')
    
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()
    
    # switch to evaluate mode
    model.eval()
    fc.eval()
        
    end = time.time()
    for i, (input, target) in enumerate(test_loader):
        target = target.to(device)
        input_var = Variable(input)
        target_var = Variable(target)

        # compute output
        with torch.no_grad():
            features = model(input_var)
            output = fc(features)
            
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        prec1, prec5 = accuracy(output.data, target.data, topk=(1, 5))
        losses.update(loss.data.item(), input.size(0))
        top1.update(prec1.item(), input.size(0))
        top5.update(prec5.item(), input.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 250 == 0:
            print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Acc@1 {top1.val:.3f} ({top1.avg:.3f})\t'
                  'Acc@5 {top5.val:.3f} ({top5.avg:.3f})'.format(
                   i, len(test_loader), batch_time=batch_time, loss=losses,
                   top1=top1, top5=top5))



    print(' * Acc@1 {top1.avg:.3f} Acc@1 Error {top1_err:.3f}\n'
          ' * Acc@5 {top5.avg:.3f} Acc@5 Error {top5_err:.3f}'
          .format(top1=top1, top1_err=100-top1.avg, top5=top5, top5_err=100-top5.avg))

    return top1, top5, losses

### Checkpoint Model

In [ ]:
%%time
for epoch in range(start_epoch, num_epochs):
    train(train_loader, model, fc, isda_criterion, optimizer, epoch)
    top1, top5, losses = validate(test_loader, model, fc, ce_criterion, epoch)

    if top1.avg > best_top1:
        print('Saving checkpoint')
        state = {
            'model_state_dict': model.module.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'fc': fc.module.state_dict(),
            'epoch': epoch,
            'loss': losses.avg, 
            'top1': top1.avg,
            'top5': top5.avg}
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/CustomModel_standard_dmla_isda_ckpt.pth')

        best_top1 = top1.avg

### Accuracy for each class

In [ ]:
# class_correct = list(0. for i in range(100))
# class_total = list(0. for i in range(100))
# with torch.no_grad():
#     for data in test_loader:
#         images, labels = data
#         outputs = model(images)
#         _, predicted = torch.max(outputs, 1)
#         c = (predicted == labels).squeeze()
#         for i in range(4):
#             label = labels[i]
#             class_correct[label] += c[i].item()
#             class_total[label] += 1


# for i in range(10):
#     print('Accuracy of %5s : %2d %%' % (
#         classes[i], 100 * class_correct[i] / class_total[i]))